In [17]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pytube import YouTube

from zipfile import ZipFile
from urllib.request import urlretrieve

from IPython.display import Image

from collections import deque

def bfs_vectorized(img, pos, labels, count):
    rows, cols = img.shape
    queue = [pos]
    total_pixel = 0
    visited = np.zeros_like(img, dtype=bool)
    
    while queue:
        x, y = queue.pop(0)
        if visited[x, y]:
            continue
        visited[x, y] = True
        labels[x, y] = count
        total_pixel += 1
        
        # Tạo mặt nạ cho các pixel xung quanh chưa được kiểm tra và có cùng giá trị
        mask = (np.abs(np.roll(img, 1, axis=0) - img) == 0) | \
               (np.abs(np.roll(img, -1, axis=0) - img) == 0) | \
               (np.abs(np.roll(img, 1, axis=1) - img) == 0) | \
               (np.abs(np.roll(img, -1, axis=1) - img) == 0)
        
        # Cập nhật hàng đợi bằng các pixel mới thỏa mãn điều kiện
        neighbors = np.argwhere(mask & ~visited)
        for nx, ny in neighbors:
            queue.append((nx, ny))
            
    return total_pixel

def bfs(img, pos, labels, count):
    queue = deque([pos])
    labels[pos] = count
    total_pixel = 0
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    while queue:
        x, y = queue.popleft()
        
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if 0 <= nx < len(img) and 0 <= ny < len(img[0]) and img[x,y] == img[nx,ny] and labels[nx,ny] == 0:
                labels[nx,ny] = count
                queue.append((nx, ny))
                total_pixel +=1

    return total_pixel

def bfs_object_detection(binary_frame, min_objPixel = 10, max_distance=50):
    rows, cols = binary_frame.shape
    labels = np.zeros((rows,cols), dtype=int)
    
    count = 1
    object = []

    for x in range(rows):
        for y in range(cols):
            if binary_frame[x,y] > 0 and labels[x,y] == 0:
                pixels = bfs(binary_frame, (x,y), labels, count)    
                if pixels >= min_objPixel:
                    xs, ys = np.where(labels == count)
                    if len(xs) > 0 and len(ys) > 0:
                        min_x, max_x = np.min(xs), np.max(xs)
                        min_y, max_y = np.min(ys), np.max(ys)
                        object.append((min_x, min_y, max_x, max_y,pixels))
                count+=1

    return object

# def colision(obj1,obj2):
#     min_x1, min_y1, max_x1, max_y1 = obj1
#     min_x2, min_y2, max_x2, max_y2 = obj2
#     return not (max_x1 < min_x2 or min_x1 > max_x2 or max_y1 < min_y2 or min_y1 > max_y2)


def tracking_colision(curr_area,pre_obj, tolerance_pixel=20):
    for i in range(len(pre_obj)):
        for j in range(i+1, len(pre_obj)):
            pre_area1 = pre_obj[i][4]
            pre_area2 = pre_obj[j][4]
            total_preArea = pre_area1 + pre_area2
            if total_preArea - tolerance_pixel <= curr_area <= total_preArea + tolerance_pixel:
                return True
    return False

def bfs_obj_collision(video_path,min_objPixel = 10, debug = False):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Không thể mở video.")
        exit()
    
    if debug:
        output_folder = 'frames_output'
        os.makedirs(output_folder, exist_ok=True)

    frame_count = 0
    previous_objects = []
    while True:
        ret, frame = cap.read()
        current_time_ms = cap.get(cv2.CAP_PROP_POS_MSEC)

        if not ret:
            print("Hết video")
            break

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray_frame, (5, 5), 0)
        _, binary = cv2.threshold(blurred, 147, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

        #Tìm đối tượng
        objects = bfs_object_detection(binary, min_objPixel)
        
        #có sự thay đổi thì tracking
        if len(objects) < len(previous_objects):
            for i in range(len(objects)):
                curr_area = objects[i][4]
                if tracking_colision(curr_area,previous_objects):
                    print(f"Chạm nhau tại: {current_time_ms}, Frame: {frame_count}")
        
        previous_objects = objects
            
        if debug:
            debug_frame = frame.copy()
            for obj in objects:
                min_x, min_y, max_x, max_y, area = obj
                cv2.rectangle(debug_frame, (min_y, min_x), (max_y, max_x), (0, 255, 0), 2)

            # Lưu khung hình có bounding boxes
            frame_filename = os.path.join(output_folder, f'frame_{frame_count:04d}.png')
            cv2.imwrite(frame_filename, debug_frame)
            print(f"Frame {frame_count}: {len(objects)}")

        frame_count+=1


In [18]:
bfs_obj_collision('video02.mp4',10,True)

Frame 0: 2
Frame 1: 2
Frame 2: 2
Frame 3: 2
Frame 4: 2
Frame 5: 2
Frame 6: 2
Frame 7: 2
Frame 8: 2
Frame 9: 2
Frame 10: 2
Frame 11: 2
Frame 12: 2
Frame 13: 2
Frame 14: 2
Frame 15: 2
Frame 16: 2
Frame 17: 2
Frame 18: 2
Frame 19: 2
Frame 20: 2
Frame 21: 2
Frame 22: 2
Frame 23: 2
Frame 24: 2
Frame 25: 2
Frame 26: 2
Frame 27: 2
Frame 28: 2
Frame 29: 2
Frame 30: 2
Frame 31: 2
Frame 32: 2
Chạm nhau tại: 1100.0, Frame: 33
Frame 33: 1
Frame 34: 2
Frame 35: 2
Frame 36: 2
Frame 37: 2
Frame 38: 2
Frame 39: 2
Frame 40: 2
Frame 41: 2
Frame 42: 2
Frame 43: 2
Frame 44: 2
Frame 45: 2
Frame 46: 2
Frame 47: 2
Frame 48: 2
Frame 49: 2
Frame 50: 2
Frame 51: 2
Frame 52: 2
Frame 53: 2
Frame 54: 2
Frame 55: 2
Frame 56: 2
Frame 57: 2
Frame 58: 2
Frame 59: 2
Frame 60: 2
Frame 61: 2
Frame 62: 2
Frame 63: 2
Frame 64: 2
Frame 65: 2
Frame 66: 2
Frame 67: 2
Frame 68: 2
Frame 69: 2
Frame 70: 2
Frame 71: 2
Frame 72: 2
Frame 73: 2
Frame 74: 2
Frame 75: 2
Frame 76: 2
Frame 77: 2
Frame 78: 2
Frame 79: 2
Frame 80: 2
Frame

In [ ]:
import cv2
import os
import numpy as np
import pytictoc

# Đường dẫn đến video
video_path = 'videoplayback.mp4'

# Thư mục lưu ảnh
output_folder = 'frames_output'
os.makedirs(output_folder, exist_ok=True)  # Tạo thư mục nếu chưa tồn tại

# Mở video
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Không thể mở video.")
    exit()

frame_count = 0  # Đếm số frame
meanTime = 00

while True:
    
    ret, frame = cap.read()
    print(frame.shape)
    if not ret:
        print("Video đã kết thúc.")
        break
    # h, w, _ = frame.shape
    # frame[0:h//4, 0:w] = (255, 255, 255)

    # Chuyển frame sang ảnh xám
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray_frame, (5, 5), 0)
    _, binary = cv2.threshold(blurred, 147, 255, cv2.THRESH_BINARY_INV)
    # edges = cv2.Canny(binary, 50, 100)
    # kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    # edges = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)


    t = pytictoc.TicToc()
    t.tic()
    # numsC , label = cv2.connectedComponents(binary, connectivity= 8)
    numsC = all_components(binary)
    # Tính toán gradient với Canny
    # grad_magnitude = np.abs(cv2.Laplacian(gray_frame, cv2.CV_64F).astype(np.uint8))
    meanTime += t.tocvalue()

    # Lưu frame vào thư mục
    frame_filename = os.path.join(output_folder, f'frame_{frame_count:04d}.png')
    cv2.imwrite(frame_filename, binary)
    print(f"Frame {frame_count}: {numsC}")

    frame_count += 1  # Tăng bộ đếm
    if(frame_count == 56):
        break

print(f"Đã lưu {frame_count} frame vào thư mục '{output_folder}'.")
meanTime /= frame_count
print(meanTime)
cap.release()
